### Unpack Graph General

In [5]:
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
import numpy as np

class DAG_Unpacking:
    def __init__(self, graph, hiddens_lst, cards_dict): 
        self.graph = nx.DiGraph()
        self.graph.add_edges_from((parent, child) for parent, child in graph)
        self.verbose = 0
        self.hiddens_lst = hiddens_lst
        self.cards = cards_dict
        self.prob_vars = {}

    def get_mvar_details(self, var):
        parents = list(self.graph.predecessors(var))
        # returns tuple(card_var, *card_parents(var)), 
        return (self.cards[var], *tuple(self.cards[parent] for parent in parents)), "_".join(parents)

    def unpack(self):
        # dicitonary of each observed variable that is both not parentless and its observed parents is not parentless
        

    def initialize_model(self):
        self.model = gp.Model("DAG Unpacking")
        self.model.reset()
        self.model.setParam('OutputFlag', self.verbose)
        self.model.params.NonConvex = 2

        ## variables
        for var in list(self.graph.nodes):
            shape, var_name = self.get_mvar_details(var)
            if not var_name:
                self.prob_vars[f'P_{var}'] = self.model.addMVar(shape=shape, vtype=GRB.CONTINUOUS, lb=0, ub=1, name=f'P_{var}')
            else:
                self.prob_vars[f'P_{var}_giv_{var_name}'] = self.model.addMVar(shape=shape, vtype=GRB.CONTINUOUS, lb=0, ub=1, name=f'P_{var}_giv_{var_name}')
        self.model.update()



# [parent, child]

# input_lst = [["X", "A"], ["l", "A"], ["l", "B"], ["Y", "B"]]
# cards_dict = {"A": 2, "B": 2, "X": 2, "Y": 2, "l": 3}
input_lst = [["Z", "A"], ["l", "A"], ["l", "B"]]
cards_dict = {"A": 2, "B": 2, "Z": 2, "l": 3}
hiddens_lst = ["l"]


example = DAG_Unpacking(input_lst, hiddens_lst, cards_dict)
# example.initialize_model()
print(example.prob_vars)

{}


In [28]:
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
import numpy as np

dag = [['Z', 'A'], ['A', 'M'], ['M', 'Y'],
       ['H_AY', 'A'], ['H_AY', 'Y'], ['H_MY', 'M'], ['H_MY', 'Y']]
cards = {'A': 2, 'M': 2, 'Y': 2, 'Z': 2}
hiddens = ['H_AY', 'H_MY']

graph = nx.DiGraph()
graph.add_edges_from((parent, child) for parent, child in dag)

# create list of observed variables
observed = [node for node in graph.nodes if node not in hiddens]

# get a list of observed variables that are parentless, as well as observed variables whose observed parents are parentless
# [node for node in observed if not list(graph.predecessors(node)) or ]

In [43]:
# [node for node in observed if list(graph.predecessors(node))]
# [i for i in list(graph.predecessors('A')) if i in observed]

# list of nodes whose observable parents have parents
[node for node in graph.nodes if node in observed and list(graph.predecessors(node)) and ]

# connect the two above:
# [not list(graph.predecessors(i)) for i in list(graph.predecessors('A')) if i in observed]

['A', 'M', 'Y']

In [50]:
dag = [['Z', 'A'], ['A', 'M'], ['M', 'Y'],
       ['H_AY', 'A'], ['H_AY', 'Y'], ['H_MY', 'M'], ['H_MY', 'Y']]
observed = ['Z', 'A', 'M', 'Y']
G = nx.DiGraph()
G.add_edges_from(dag)

In [69]:
# (1) parentless observed nodes
[node for node in observed if not list(G.predecessors(node))]
# (2) observed nodes whose parents are parentless
for node in observed:
    if list(G.predecessors(node)):
        # print node, and whether it has an observed parent
        # print(node, [not list(G.predecessors(i)) for i in list(G.predecessors(node)) if i in observed])
        print(node, [parent for parent in list(G.predecessors(node)) if parent in observed])

A ['Z']
M ['A']
Y ['M']


In [45]:
import networkx as nx


dag = [['Z', 'A'], ['A', 'M'], ['M', 'Y'],
       ['H_AY', 'A'], ['H_AY', 'Y'], ['H_MY', 'M'], ['H_MY', 'Y']]
cards = {'A': 2, 'M': 2, 'Y': 2, 'Z': 2}

G = nx.DiGraph()
G.add_edges_from(dag)
observed = ['Z', 'A', 'M', 'Y']

# Function to get observed parents of a node
def get_observed_parents(node, observed, G):
    return [parent for parent in G.predecessors(node) if parent in observed]

# List comprehension to get the desired nodes
filtered_nodes = [
    node for node in G.nodes
    if node not in observed and
    all(
        len(list(G.predecessors(parent))) > 0
        for parent in get_observed_parents(node, observed, G)
    )
]

print(filtered_nodes)

['H_AY', 'H_MY']


In [27]:
# create list of observed variables
observed = [node for node in graph.nodes if node not in hiddens]

# get a list of observed variables that are parentless, as well as observed variables whose observed parents are parentless
# observed_parentless = [node for node in observed if not list(graph.predecessors(node)) or all([parent in parentless for parent in graph.predecessors(node)])]
# print(observed_parentless)

# parentless = [node for node in graph.nodes if not list(graph.predecessors(node)) or all([parent in parentless for parent in graph.predecessors(node)])]
